In [1]:
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.automl import H2OAutoML, get_leaderboard
import datatable as DT
import pandas as pd

In [2]:
#Load into Python Pandas and manipulate (i.e. drop cols that leak)
df = pd.read_csv("./data/finalapi.csv")

In [3]:
df.describe()

,AGENCY_ID,PRIMARY_AGENCY_ID,STAT_PROFILE_DATE_YEAR,RETENTION_POLY_QTY,POLY_INFORCE_QTY,PREV_POLY_INFORCE_QTY,NB_WRTN_PREM_AMT,WRTN_PREM_AMT,PREV_WRTN_PREM_AMT,PRD_ERND_PREM_AMT,...,PL_BOUND_CT_ELINKS,PL_QUO_CT_ELINKS,PL_BOUND_CT_PLRANK,PL_QUO_CT_PLRANK,PL_BOUND_CT_eQTte,PL_QUO_CT_eQTte,PL_BOUND_CT_APPLIED,PL_QUO_CT_APPLIED,PL_BOUND_CT_TRANSACTNOW,PL_QUO_CT_TRANSACTNOW
count,213328.000000,213328.000000,213328.000000,213328.000000,213328.000000,213328.000000,213328.000000,2.133280e+05,2.133280e+05,2.133280e+05,...,213328.000000,213328.000000,213328.000000,213328.000000,213328.000000,213328.000000,213328.000000,213328.000000,213328.000000,213328.000000
mean,4978.964158,21840.982206,2010.191738,158.634525,175.616919,6694.153369,2243.651368,1.963268e+04,2.600812e+04,1.945872e+04,...,26928.162206,26969.472127,26931.800401,27039.756497,26963.478081,27091.403323,26928.081002,26933.596284,26927.300134,26927.453377
std,2928.027642,36410.355440,3.129108,718.766295,774.640860,24639.586044,9976.801667,6.644395e+04,7.008093e+04,6.635115e+04,...,44357.460628,44333.186665,44355.256891,44291.273197,44336.059501,44258.970692,44357.509786,44354.180214,44357.983443,44357.890447
min,3.000000,3.000000,2005.000000,0.000000,0.000000,0.000000,0.000000,-2.027776e+05,-1.861810e+05,-1.643490e+05,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2366.000000,2788.000000,2008.000000,0.000000,0.000000,0.000000,0.000000,2.027500e+00,1.480000e+02,7.600000e+01,...,0.000000,0.000000,0.000000,0.000000,11.000000,66.000000,0.000000,0.000000,0.000000,0.000000
50%,4976.000000,6110.000000,2010.000000,0.000000,0.000000,0.000000,0.000000,1.143620e+03,1.781000e+03,1.180490e+03,...,0.000000,0.000000,0.000000,0.000000,51.000000,258.000000,0.000000,0.000000,0.000000,0.000000
75%,7589.000000,9285.000000,2013.000000,24.000000,30.000000,72.000000,485.000000,8.358695e+03,1.477315e+04,8.231562e+03,...,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000
max,9998.000000,99999.000000,2015.000000,21979.000000,22968.000000,99999.000000,556930.220000,1.715742e+06,1.904570e+06,1.780498e+06,...,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000


In [4]:
#Drop potential leaks
df = df.drop(columns=['RETENTION_POLY_QTY', 'POLY_INFORCE_QTY', 'PREV_POLY_INFORCE_QTY', 'NB_WRTN_PREM_AMT', 'PREV_WRTN_PREM_AMT', 'RETENTION_RATIO', 'PRD_ERND_PREM_AMT'])

In [5]:
df.describe()

,AGENCY_ID,PRIMARY_AGENCY_ID,STAT_PROFILE_DATE_YEAR,WRTN_PREM_AMT,PRD_INCRD_LOSSES_AMT,MONTHS,LOSS_RATIO,LOSS_RATIO_3YR,GROWTH_RATE_3YR,AGENCY_APPOINTMENT_YEAR,...,PL_BOUND_CT_ELINKS,PL_QUO_CT_ELINKS,PL_BOUND_CT_PLRANK,PL_QUO_CT_PLRANK,PL_BOUND_CT_eQTte,PL_QUO_CT_eQTte,PL_BOUND_CT_APPLIED,PL_QUO_CT_APPLIED,PL_BOUND_CT_TRANSACTNOW,PL_QUO_CT_TRANSACTNOW
count,213328.000000,213328.000000,213328.000000,2.133280e+05,2.133280e+05,213328.000000,213328.000000,213328.000000,213328.000000,213328.000000,...,213328.000000,213328.000000,213328.000000,213328.000000,213328.000000,213328.000000,213328.000000,213328.000000,213328.000000,213328.000000
mean,4978.964158,21840.982206,2010.191738,1.963268e+04,1.107296e+04,10.370420,24926.569482,23024.118682,42688.285812,4578.353517,...,26928.162206,26969.472127,26931.800401,27039.756497,26963.478081,27091.403323,26928.081002,26933.596284,26927.300134,26927.453377
std,2928.027642,36410.355440,3.129108,6.644395e+04,6.947273e+04,3.008576,43257.880847,42098.349517,49462.194579,15723.090320,...,44357.460628,44333.186665,44355.256891,44291.273197,44336.059501,44258.970692,44357.509786,44354.180214,44357.983443,44357.890447
min,3.000000,3.000000,2005.000000,-2.027776e+05,-5.000000e+06,1.000000,-756.228571,-7215.007215,-1.000000,1935.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2366.000000,2788.000000,2008.000000,2.027500e+00,0.000000e+00,10.000000,0.000000,0.271818,-0.038417,1978.000000,...,0.000000,0.000000,0.000000,0.000000,11.000000,66.000000,0.000000,0.000000,0.000000,0.000000
50%,4976.000000,6110.000000,2010.000000,1.143620e+03,0.000000e+00,12.000000,0.000000,0.543219,0.141791,1990.000000,...,0.000000,0.000000,0.000000,0.000000,51.000000,258.000000,0.000000,0.000000,0.000000,0.000000
75%,7589.000000,9285.000000,2013.000000,8.358695e+03,0.000000e+00,12.000000,85.958571,2.726085,99999.000000,2002.000000,...,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000
max,9998.000000,99999.000000,2015.000000,1.715742e+06,6.241231e+06,12.000000,99999.000000,99999.000000,99999.000000,99999.000000,...,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000


In [6]:
#Write out CSV for Driverless AI
df.to_csv('./data/finalapi_processed.csv')

In [7]:
#Initialize H2O Cluster
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_152-release"; OpenJDK Runtime Environment (build 1.8.0_152-release-1056-b12); OpenJDK 64-Bit Server VM (build 25.152-b12, mixed mode)
  Starting server from /Users/thomasott/opt/anaconda3/envs/py37/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/9c/tqhnrz3x207bf8pzjxjhkm100000gn/T/tmpmhd8hw35
  JVM stdout: /var/folders/9c/tqhnrz3x207bf8pzjxjhkm100000gn/T/tmpmhd8hw35/h2o_thomasott_started_from_python.out
  JVM stderr: /var/folders/9c/tqhnrz3x207bf8pzjxjhkm100000gn/T/tmpmhd8hw35/h2o_thomasott_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.2
H2O cluster version age:,1 day
H2O cluster name:,H2O_from_python_thomasott_2ah9cg
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [8]:
# Import a proessed training dataset
train = h2o.import_file('./data/finalapi_processed.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [9]:
# Identify predictors and response
x = train.columns
y = 'WRTN_PREM_AMT'
x.remove(y)

In [10]:
#Setup GBM model with stopping metric = MAE
premium_gbm = H2OGradientBoostingEstimator(model_id = "premium_gbm", ntrees=100, 
                                           learn_rate=0.001, max_depth=10, stopping_metric='MAE', 
                                           categorical_encoding='auto', fold_column = 'STATE_ABBR')

In [11]:
## Building H2O GBM Model:
premium_gbm.train(x = x, y = y, training_frame=train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [12]:
#Get MAE performances for Train
train_performance = premium_gbm.model_performance(train)

In [13]:
train_performance.mae()

26331.319678386546

In [14]:
aml = H2OAutoML(max_runtime_secs=3600, seed=1234, stopping_metric="MAE", sort_metric='MAE')
aml.train(x=x, y=y, training_frame=train, fold_column = 'STATE_ABBR')

AutoML progress: |
09:39:50.160: Fold column STATE_ABBR will be used for cross-validation. nfolds parameter will be ignored.

████████████████████████████████████████████████████████| 100%


In [15]:
# AutoML Leaderboard
lb = aml.leaderboard

In [16]:
# Optionally edd extra model information to the leaderboard
lb = get_leaderboard(aml)

In [17]:
# Print all rows (instead of default 10 rows)
lb.head(rows=lb.nrows)

model_id,mae,mean_residual_deviance,rmse,mse,rmsle
XGBoost_1_AutoML_20200122_093950,4203.44,1.6238e+08,12742.8,1.6238e+08,nan
StackedEnsemble_AllModels_AutoML_20200122_093950,4381.55,1.37744e+08,11736.4,1.37744e+08,nan
XGBoost_2_AutoML_20200122_093950,5082.23,2.10762e+08,14517.7,2.10762e+08,nan


In [19]:
#Load in DAI processed Features and retraing AutoML
train_features = h2o.import_file('./data/finalapi_processed_features.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [21]:
# Identify predictors and response
x = train_features.columns
y = 'WRTN_PREM_AMT'
x.remove(y)

In [24]:
print(x)

['109_InteractionDiv:LOSS_RATIO:PRD_INCRD_LOSSES_AMT', '189_InteractionSub:LOSS_RATIO:PRD_INCRD_LOSSES_AMT', '52_NumToCatTE:LOSS_RATIO:PRD_INCRD_LOSSES_AMT.0', '55_NumCatTE:LOSS_RATIO:PRD_INCRD_LOSSES_AMT:PROD_ABBR.0', '60_NumToCatTE:LOSS_RATIO:PRD_INCRD_LOSSES_AMT.0']


In [25]:
aml_features = H2OAutoML(max_runtime_secs=3600, seed=1234, stopping_metric="MAE", sort_metric='MAE')
aml_features.train(x=x, y=y, training_frame=train_features)

AutoML progress: |██████████████████████████████████████████████████████Failed polling AutoML progress log: HTTP 500 Server Error:
Server error java.lang.ArrayIndexOutOfBoundsException:
  Error: 77
  Request: None

██| 100%


In [26]:
# AutoML Leaderboard
lb_features = aml_features.leaderboard

# Optionally edd extra model information to the leaderboard
lb_features = get_leaderboard(aml_features)

# Print all rows (instead of default 10 rows)
lb_features.head(rows=lb.nrows)

model_id,mae,mean_residual_deviance,rmse,mse,rmsle
XGBoost_grid__1_AutoML_20200122_104741_model_2,2411.19,5.32759e+07,7299.04,5.32759e+07,nan
StackedEnsemble_AllModels_AutoML_20200122_104741,2418.33,4.82553e+07,6946.6,4.82553e+07,nan
XGBoost_grid__1_AutoML_20200122_104741_model_1,2445.39,4.99113e+07,7064.79,4.99113e+07,nan
